# 🧬 Heme Binder Diffusion Pipeline for Google Colab

This notebook provides a simplified version of the heme binding protein design pipeline optimized for Google Colab Pro.

## ⚠️ Important Notes:
- **GPU Required**: Make sure you have GPU enabled (Runtime > Change runtime type > GPU)
- **Time Limits**: Colab Pro has 24h session limits - save progress frequently
- **Storage**: Use Google Drive for persistent storage
- **Resources**: Monitor GPU/RAM usage to avoid session termination

## 🚀 Pipeline Overview:
1. **Setup Environment** - Install dependencies and download models
2. **Generate Backbones** - Use RFdiffusion to create protein scaffolds
3. **Design Sequences** - Use ProteinMPNN for sequence design
4. **Predict Structures** - Use AlphaFold2 for structure prediction
5. **Optimize Binding** - Use LigandMPNN for binding site optimization
6. **Analyze Results** - Score and filter final designs

## 🔧 Step 1: Environment Setup

This will take 15-30 minutes depending on your internet connection.

In [ ]:
# Check if GPU is available
import torch
if torch.cuda.is_available():
    print(f"✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ No GPU detected! Please enable GPU in Runtime > Change runtime type")
    raise RuntimeError("GPU required for this pipeline")

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Create project directory in Drive
import os
DRIVE_DIR = "/content/drive/MyDrive/heme_binder_diffusion"
os.makedirs(DRIVE_DIR, exist_ok=True)
print(f"✅ Project directory: {DRIVE_DIR}")

In [ ]:
# Clone repository and setup
import subprocess
import os

# Clone the repository
if not os.path.exists("/content/heme_binder_diffusion"):
    !git clone https://github.com/ikalvet/heme_binder_diffusion.git /content/heme_binder_diffusion
    
os.chdir("/content/heme_binder_diffusion")

# Initialize submodules
!git submodule init
!git submodule update

print("✅ Repository cloned and submodules initialized")

In [ ]:
# Download and run setup script
import requests
import os

# Download setup script (you'll need to upload this to a accessible URL)
setup_script_url = "https://raw.githubusercontent.com/YOUR_USERNAME/heme_binder_diffusion/main/colab_setup.py"

# For now, we'll create the script inline
setup_script = '''
import subprocess
import sys
import os

# Install core dependencies
print("Installing core dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "pip"])

# Core packages
packages = [
    "torch>=1.12.0",
    "numpy>=1.21.0",
    "pandas>=1.3.0",
    "matplotlib>=3.5.0",
    "seaborn",
    "biopython>=1.79",
    "prody>=2.0",
    "py3dmol>=1.8",
    "tqdm",
    "requests"
]

for package in packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], check=True)
        print(f"✅ {package}")
    except:
        print(f"❌ {package}")

# Install JAX
try:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "jax[cuda12_pip]", "-f", 
                   "https://storage.googleapis.com/jax-releases/jax_cuda_releases.html"], check=True)
    print("✅ JAX with CUDA")
except:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "jax", "jaxlib"])
    print("✅ JAX (CPU version)")

print("✅ Setup completed!")
'''

# Write and execute setup script
with open('colab_setup.py', 'w') as f:
    f.write(setup_script)

exec(open('colab_setup.py').read())

In [ ]:
# Download essential models (this will take time!)
import os
import requests
from tqdm import tqdm

def download_file(url, filepath):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filepath, 'wb') as f:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc=f"Downloading {os.path.basename(filepath)}") as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

# Create models directory
models_dir = "/content/models"
os.makedirs(models_dir, exist_ok=True)

# Download ProteinMPNN models (smaller and essential)
print("📥 Downloading ProteinMPNN models...")
mpnn_dir = f"{models_dir}/proteinmpnn"
os.makedirs(mpnn_dir, exist_ok=True)

# Note: You'll need to check these URLs and adjust them
models_to_download = {
    "proteinmpnn_v_48_020.pt": "https://files.ipd.uw.edu/pub/training_sets/proteinmpnn_v_48_020.pt",
    "ligandmpnn_v_32_010_25.pt": "https://files.ipd.uw.edu/pub/training_sets/ligandmpnn_v_32_010_25.pt"
}

for filename, url in models_to_download.items():
    filepath = f"{mpnn_dir}/{filename}"
    if not os.path.exists(filepath):
        try:
            download_file(url, filepath)
            print(f"✅ {filename}")
        except Exception as e:
            print(f"❌ Failed to download {filename}: {e}")
    else:
        print(f"✅ {filename} already exists")

print("✅ Essential models downloaded!")
print("⚠️ Note: AlphaFold2 and RFdiffusion models are very large (~4GB each)")
print("   They will be downloaded on-demand when needed")

## ⚙️ Configuration and Helper Functions

In [ ]:
# Setup configuration
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Paths
BASE_DIR = "/content/heme_binder_diffusion"
MODELS_DIR = "/content/models"
WORK_DIR = "/content/work"
DRIVE_DIR = "/content/drive/MyDrive/heme_binder_diffusion"

# Create working directory
os.makedirs(WORK_DIR, exist_ok=True)

# Add scripts to Python path
sys.path.append(f"{BASE_DIR}/scripts/utils")
sys.path.append(f"{BASE_DIR}/lib/LigandMPNN")

# Project settings
PROJECT_NAME = "colab_heme_design"
LIGAND = "HBA"
N_DESIGNS = 3  # Reduced for Colab
T_STEPS = 100  # Reduced for Colab

print(f"✅ Configuration set up")
print(f"📁 Base directory: {BASE_DIR}")
print(f"📁 Work directory: {WORK_DIR}")
print(f"📁 Drive backup: {DRIVE_DIR}")

In [ ]:
# Helper functions
import subprocess
import time
import json
from IPython.display import HTML, display
import py3dmol

def run_command(cmd, description=""):
    """Run a shell command and return result"""
    print(f"🔄 {description}")
    print(f"Command: {cmd}")
    
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ {description} completed")
        return result.stdout
    else:
        print(f"❌ {description} failed")
        print(f"Error: {result.stderr}")
        return None

def save_to_drive(local_path, drive_subdir=""):
    """Save files to Google Drive"""
    drive_path = f"{DRIVE_DIR}/{drive_subdir}"
    os.makedirs(drive_path, exist_ok=True)
    
    if os.path.isfile(local_path):
        shutil.copy2(local_path, drive_path)
    else:
        shutil.copytree(local_path, f"{drive_path}/{os.path.basename(local_path)}", dirs_exist_ok=True)
    
    print(f"💾 Saved to Drive: {drive_path}")

def visualize_pdb(pdb_path, width=400, height=400):
    """Visualize PDB structure"""
    with open(pdb_path, 'r') as f:
        pdb_data = f.read()
    
    view = py3dmol.view(width=width, height=height)
    view.addModel(pdb_data, 'pdb')
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.setStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})
    view.zoomTo()
    view.show()
    
    return view

def monitor_resources():
    """Monitor GPU and RAM usage"""
    import torch
    import psutil
    
    # GPU usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        gpu_used = torch.cuda.memory_allocated(0) / 1024**3
        gpu_percent = (gpu_used / gpu_memory) * 100
        print(f"🖥️  GPU: {gpu_used:.1f}/{gpu_memory:.1f} GB ({gpu_percent:.1f}%)")
    
    # RAM usage
    ram = psutil.virtual_memory()
    ram_used = ram.used / 1024**3
    ram_total = ram.total / 1024**3
    ram_percent = ram.percent
    print(f"💾 RAM: {ram_used:.1f}/{ram_total:.1f} GB ({ram_percent:.1f}%)")
    
    # Disk usage
    disk = psutil.disk_usage('/')
    disk_used = disk.used / 1024**3
    disk_total = disk.total / 1024**3
    disk_percent = (disk_used / disk_total) * 100
    print(f"💿 Disk: {disk_used:.1f}/{disk_total:.1f} GB ({disk_percent:.1f}%)")

print("✅ Helper functions loaded")

## 🧬 Step 2: Simplified Pipeline Execution

This is a streamlined version that focuses on the core functionality.

In [ ]:
# Check resources before starting
print("📊 Current resource usage:")
monitor_resources()

# Check available input files
input_files = glob.glob(f"{BASE_DIR}/input/*.pdb")
print(f"\n📁 Found {len(input_files)} input PDB files:")
for f in input_files:
    print(f"  - {os.path.basename(f)}")

if len(input_files) == 0:
    print("⚠️ No input files found! Please check the input directory.")
else:
    print("✅ Ready to start pipeline")

In [ ]:
# Visualize input structure
if input_files:
    input_pdb = input_files[0]  # Use first file as example
    print(f"📊 Visualizing input structure: {os.path.basename(input_pdb)}")
    
    view = visualize_pdb(input_pdb)
    
    # Also show some basic statistics
    with open(input_pdb, 'r') as f:
        lines = f.readlines()
    
    atom_lines = [l for l in lines if l.startswith('ATOM')]
    hetatm_lines = [l for l in lines if l.startswith('HETATM')]
    
    print(f"📈 Structure statistics:")
    print(f"  - Protein atoms: {len(atom_lines)}")
    print(f"  - Ligand atoms: {len(hetatm_lines)}")
    print(f"  - Total lines: {len(lines)}")

### 🧩 Step 2.1: ProteinMPNN Sequence Design

We'll start with sequence design using ProteinMPNN (skipping RFdiffusion for now due to model size).

In [ ]:
# Setup ProteinMPNN run
import os
import json

# Create output directory
mpnn_dir = f"{WORK_DIR}/proteinmpnn"
os.makedirs(mpnn_dir, exist_ok=True)
os.chdir(mpnn_dir)

# ProteinMPNN parameters
temperatures = [0.1, 0.2, 0.3]
n_sequences = 3  # Reduced for Colab
omit_AAs = "CM"

# Create a simple mask (no fixed residues for this demo)
mask_dict = {}
for input_file in input_files:
    basename = os.path.basename(input_file).replace('.pdb', '')
    mask_dict[basename] = {}

with open('mask.jsonl', 'w') as f:
    json.dump(mask_dict, f)

print(f"✅ ProteinMPNN setup complete")
print(f"📊 Will generate {len(temperatures) * n_sequences} sequences per structure")

In [ ]:
# Run ProteinMPNN
import subprocess
import sys

# Check if ProteinMPNN script exists
mpnn_script = f"{BASE_DIR}/lib/LigandMPNN/run.py"
if not os.path.exists(mpnn_script):
    print(f"❌ ProteinMPNN script not found: {mpnn_script}")
    print("Please make sure submodules are properly initialized")
else:
    print(f"✅ ProteinMPNN script found")
    
    # Run for each temperature and input file
    for temp in temperatures:
        for input_file in input_files[:1]:  # Only first file for demo
            cmd = f"""{sys.executable} {mpnn_script} \
                --model_type protein_mpnn \
                --ligand_mpnn_use_atom_context 0 \
                --fixed_residues_multi mask.jsonl \
                --out_folder ./ \
                --number_of_batches {n_sequences} \
                --temperature {temp} \
                --omit_AA {omit_AAs} \
                --pdb_path {input_file} \
                --checkpoint_protein_mpnn {MODELS_DIR}/proteinmpnn/proteinmpnn_v_48_020.pt"""
            
            print(f"🔄 Running ProteinMPNN with temperature {temp}...")
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            
            if result.returncode == 0:
                print(f"✅ Completed T={temp}")
            else:
                print(f"❌ Failed T={temp}")
                print(f"Error: {result.stderr}")
                
    # Check outputs
    fasta_files = glob.glob("seqs/*.fa")
    print(f"\n📊 Generated {len(fasta_files)} FASTA files")
    
    if fasta_files:
        # Save to Drive
        save_to_drive(mpnn_dir, "proteinmpnn_results")
        print("✅ ProteinMPNN results saved to Drive")
    else:
        print("⚠️ No FASTA files generated")

### 🔬 Step 2.2: AlphaFold2 Structure Prediction

We'll use ColabFold for faster AlphaFold2 predictions.

In [ ]:
# Install ColabFold if not already installed
try:
    import colabfold
    print("✅ ColabFold already installed")
except ImportError:
    print("📦 Installing ColabFold...")
    !pip install -q colabfold[alphafold]
    print("✅ ColabFold installed")

# Alternative: Use the original AlphaFold2 script if available
af2_script = f"{BASE_DIR}/scripts/af2/af2.py"
if os.path.exists(af2_script):
    print("✅ Original AlphaFold2 script found")
    use_original_af2 = True
else:
    print("⚠️ Using ColabFold instead of original AlphaFold2")
    use_original_af2 = False

In [ ]:
# Prepare sequences for AlphaFold2
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

# Create AF2 directory
af2_dir = f"{WORK_DIR}/alphafold2"
os.makedirs(af2_dir, exist_ok=True)
os.chdir(af2_dir)

# Parse ProteinMPNN outputs
sequences = {}
fasta_files = glob.glob(f"{mpnn_dir}/seqs/*.fa")

for fasta_file in fasta_files:
    with open(fasta_file, 'r') as f:
        for record in SeqIO.parse(f, 'fasta'):
            seq_id = f"{os.path.basename(fasta_file).replace('.fa', '')}_{record.id}"
            sequences[seq_id] = str(record.seq)

print(f"📊 Found {len(sequences)} sequences for AlphaFold2 prediction")

# Create combined FASTA file
records = []
for seq_id, seq in sequences.items():
    records.append(SeqRecord(Seq(seq), id=seq_id, description=""))

with open('input_sequences.fasta', 'w') as f:
    SeqIO.write(records, f, 'fasta')

print(f"✅ Created input_sequences.fasta with {len(records)} sequences")

In [ ]:
# Run AlphaFold2 predictions
import subprocess
import time

if use_original_af2:
    # Use original AlphaFold2 script
    cmd = f"""{sys.executable} {af2_script} \
        --af-nrecycles 3 \
        --af-models 4 \
        --fasta input_sequences.fasta \
        --scorefile af2_scores.csv"""
    
    print("🔄 Running AlphaFold2 predictions...")
    print("⏰ This may take 10-30 minutes depending on sequence length and number")
    
    start_time = time.time()
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    end_time = time.time()
    
    if result.returncode == 0:
        print(f"✅ AlphaFold2 completed in {(end_time - start_time)/60:.1f} minutes")
    else:
        print(f"❌ AlphaFold2 failed")
        print(f"Error: {result.stderr}")
        
else:
    # Use ColabFold
    print("🔄 Running ColabFold predictions...")
    print("⏰ This may take 5-15 minutes")
    
    # ColabFold command
    cmd = f"colabfold_batch input_sequences.fasta ./predictions --num-models 1 --num-recycles 3"
    
    start_time = time.time()
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    end_time = time.time()
    
    if result.returncode == 0:
        print(f"✅ ColabFold completed in {(end_time - start_time)/60:.1f} minutes")
    else:
        print(f"❌ ColabFold failed")
        print(f"Error: {result.stderr}")

# Check outputs
pdb_files = glob.glob("*.pdb") + glob.glob("predictions/*.pdb")
print(f"\n📊 Generated {len(pdb_files)} PDB files")

if pdb_files:
    # Save to Drive
    save_to_drive(af2_dir, "alphafold2_results")
    print("✅ AlphaFold2 results saved to Drive")
else:
    print("⚠️ No PDB files generated")

## 📊 Step 3: Analysis and Visualization

Analyze the results and visualize the best structures.

In [ ]:
# Analyze AlphaFold2 results
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load confidence scores if available
confidence_files = glob.glob("*_confidence.json") + glob.glob("predictions/*_confidence.json")
pdb_files = glob.glob("*.pdb") + glob.glob("predictions/*.pdb")

print(f"📊 Analyzing {len(pdb_files)} structures")

# Basic analysis
results = []
for pdb_file in pdb_files:
    # Get basic info
    basename = os.path.basename(pdb_file)
    
    # Count atoms
    with open(pdb_file, 'r') as f:
        lines = f.readlines()
    
    atom_lines = [l for l in lines if l.startswith('ATOM')]
    
    # Extract confidence scores if available (ColabFold format)
    confidence_scores = []
    for line in atom_lines:
        if len(line) > 60:
            try:
                confidence = float(line[60:66].strip())
                confidence_scores.append(confidence)
            except:
                pass
    
    avg_confidence = np.mean(confidence_scores) if confidence_scores else 0
    
    results.append({
        'filename': basename,
        'n_atoms': len(atom_lines),
        'avg_confidence': avg_confidence,
        'path': pdb_file
    })

# Create results DataFrame
results_df = pd.DataFrame(results)

if len(results_df) > 0:
    print("\n📈 Results Summary:")
    print(results_df.describe())
    
    # Plot confidence scores
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.hist(results_df['avg_confidence'], bins=20, alpha=0.7)
    plt.xlabel('Average Confidence Score')
    plt.ylabel('Count')
    plt.title('Distribution of Confidence Scores')
    
    plt.subplot(1, 2, 2)
    plt.scatter(results_df['n_atoms'], results_df['avg_confidence'], alpha=0.7)
    plt.xlabel('Number of Atoms')
    plt.ylabel('Average Confidence Score')
    plt.title('Confidence vs Structure Size')
    
    plt.tight_layout()
    plt.show()
    
    # Save results
    results_df.to_csv('analysis_results.csv', index=False)
    print("✅ Analysis results saved")
else:
    print("⚠️ No structures to analyze")

In [ ]:
# Visualize best structures
if len(results_df) > 0:
    # Sort by confidence score
    best_structures = results_df.sort_values('avg_confidence', ascending=False).head(3)
    
    print("🏆 Top 3 structures by confidence:")
    print(best_structures[['filename', 'avg_confidence', 'n_atoms']])
    
    # Visualize the best structure
    best_pdb = best_structures.iloc[0]['path']
    print(f"\n📊 Visualizing best structure: {os.path.basename(best_pdb)}")
    
    view = visualize_pdb(best_pdb, width=600, height=400)
    
    # Show structure info
    print(f"\n📈 Structure Details:")
    print(f"  - Filename: {best_structures.iloc[0]['filename']}")
    print(f"  - Confidence: {best_structures.iloc[0]['avg_confidence']:.2f}")
    print(f"  - Atoms: {best_structures.iloc[0]['n_atoms']}")
else:
    print("⚠️ No structures available for visualization")

## 💾 Step 4: Save Results and Cleanup

In [ ]:
# Save all results to Google Drive
import shutil
import zipfile
from datetime import datetime

# Create timestamped results directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_dir = f"{DRIVE_DIR}/results_{timestamp}"
os.makedirs(results_dir, exist_ok=True)

# Copy all important files
files_to_save = {
    'proteinmpnn_results': f"{WORK_DIR}/proteinmpnn",
    'alphafold2_results': f"{WORK_DIR}/alphafold2",
    'analysis_results.csv': 'analysis_results.csv'
}

for name, path in files_to_save.items():
    if os.path.exists(path):
        dest_path = f"{results_dir}/{name}"
        if os.path.isdir(path):
            shutil.copytree(path, dest_path, dirs_exist_ok=True)
        else:
            shutil.copy2(path, dest_path)
        print(f"✅ Saved {name}")

# Create a summary report
summary_report = f"""
# Heme Binder Diffusion Pipeline Results
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Pipeline Summary
- Project: {PROJECT_NAME}
- Ligand: {LIGAND}
- Input files: {len(input_files)}
- Generated sequences: {len(sequences)}
- Predicted structures: {len(pdb_files)}

## Best Results
""";

if len(results_df) > 0:
    for idx, row in best_structures.iterrows():
        summary_report += f"\n- {row['filename']}: Confidence {row['avg_confidence']:.2f}"

summary_report += f"""

## Files Generated
- ProteinMPNN sequences: {len(fasta_files)} FASTA files
- AlphaFold2 structures: {len(pdb_files)} PDB files
- Analysis results: analysis_results.csv

## Next Steps
1. Review the top-confidence structures
2. Perform binding site optimization with LigandMPNN
3. Run molecular dynamics simulations
4. Experimental validation
"""

with open(f"{results_dir}/README.md", 'w') as f:
    f.write(summary_report)

print(f"\n✅ All results saved to: {results_dir}")
print(f"📊 Summary report created: {results_dir}/README.md")

In [ ]:
# Final resource check and cleanup
print("🧹 Final cleanup and resource check:")
monitor_resources()

# Optional: Clean up large temporary files
cleanup_choice = input("\nClean up temporary files? (y/n): ")
if cleanup_choice.lower() == 'y':
    import shutil
    
    # Remove work directory (keeping only Drive backup)
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)
        print(f"🧹 Cleaned up {WORK_DIR}")
    
    # Clear GPU cache
    import torch
    torch.cuda.empty_cache()
    print("🧹 Cleared GPU cache")
    
    monitor_resources()

print("\n🎉 Pipeline completed successfully!")
print(f"📁 Results saved in Google Drive: {results_dir}")
print("\n📋 Summary:")
print(f"  - Input structures: {len(input_files)}")
print(f"  - Generated sequences: {len(sequences)}")
print(f"  - Predicted structures: {len(pdb_files)}")
if len(results_df) > 0:
    print(f"  - Best confidence score: {results_df['avg_confidence'].max():.2f}")
    print(f"  - Average confidence: {results_df['avg_confidence'].mean():.2f}")
    
print("\n🚀 Next steps:")
print("  1. Download results from Google Drive")
print("  2. Analyze structures in detail")
print("  3. Run binding site optimization")
print("  4. Prepare for experimental validation")